In [11]:
import numpy as np

ImportError: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.

Original error was: cannot import name 'multiarray'


In [3]:
# For getting how long each group of customers would spend
np.random.uniform(30,150) # Generate one random number between 30 and 150 under uniform distribution

NameError: name 'np' is not defined

In [4]:
class Tables:
    def __init__(self, num, size):
        self.num=num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        self.currentStatus = None # Is the table occupied or not.
        self.time_remaining = 0 
        
    def busy(self):
        if self.currentStatus != None:
            return True
        else:
            return False
    
    def startNext(self, newtask):
        self.currentStatus = newtask
        self.time_remaining = newtask.time_needs() # here maybe generate a random time from 10 - 60 mins using generat

In [9]:
def simulation(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Tables(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Tables(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Tables(i+number_tables_4+number_tables_6,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [10]:
table_2_list, table_4_list, table_6_list = simulation(6,4,2)

In [7]:
table_2_list[0].num

0

In [8]:
table_4_list[0].num

2

In [ ]:
def mod_pert_random(low, likely, high, confidence=4, samples=10000):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta